In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_Fatigue_Syndrome"
cohort = "GSE39684"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_Fatigue_Syndrome"
in_cohort_dir = "../../input/GEO/Chronic_Fatigue_Syndrome/GSE39684"

# Output paths
out_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/GSE39684.csv"
out_gene_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE39684.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/clinical_data/GSE39684.csv"
json_path = "../../output/preprocess/Chronic_Fatigue_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comprehensive Investigation of Archival and Prospectively Collected Samples Reveals No Association of the XMRV Gammaretrovirus with Prostate Cancer"
!Series_summary	"XMRV, or xenotropic murine leukemia virus (MLV)-related virus, is a novel gammaretrovirus originally identified in studies that analyzed tissue from prostate cancer patients in 2006 and blood from patients with chronic fatigue syndrome (CFS) in 2009. However, a large number of subsequent studies failed to confirm a link between XMRV infection and CFS or prostate cancer. On the contrary, recent evidence indicates that XMRV is a contaminant originating from the recombination of two mouse endogenous retroviruses during passaging of a prostate tumor xenograft (CWR22) in mice, generating laboratory-derived cell lines that are XMRV-infected. To confirm or refute an association between XMRV and prostate cancer, we analyzed prostate cancer tissues and plasma from a prospectively collected coh

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on background info, this is a microarray study looking for XMRV virus
# It's using the Virochip microarray which contains viral probes, not gene expression data
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# From the sample characteristics, we need to identify which rows contain our variables

# 2.1 Data Availability
# Looking at the sample characteristics dictionary:
# Row 0 has 'cell line: 22Rv1', 'tissue: prostate biopsy'
# Row 1 has 'polya enrichment: yes', 'polya enrichment: no', 'cohort: 2006', 'cohort: 2012'
# Row 2 has [nan, 'polya enrichment: yes', 'polya enrichment: no']

# For the trait (Chronic Fatigue Syndrome):
# The background information mentions studying chronic fatigue syndrome (CFS) but no specific row 
# in the sample characteristics identifies CFS status. The study is about analyzing XMRV virus 
# in prostate cancer tissue, not about CFS.
trait_row = None  # No data available for Chronic Fatigue Syndrome

# For age:
# No age information is provided in the sample characteristics
age_row = None

# For gender:
# Since this is a prostate cancer study, all subjects are male.
# This is a constant feature (all subjects have the same value), so we consider it not available
gender_row = None

# 2.2 Data Type Conversion
# Define conversion functions for each variable
def convert_trait(value: str) -> Optional[int]:
    """Convert trait values to binary format (0 or 1)."""
    if pd.isna(value):
        return None
    
    value_part = value.split(':')[-1].strip().lower() if ':' in value else value.lower().strip()
    
    # For Chronic Fatigue Syndrome, but since trait_row is None, this function won't be used
    if 'cfs' in value_part or 'chronic fatigue' in value_part:
        return 1
    elif 'control' in value_part or 'healthy' in value_part:
        return 0
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age values to continuous format."""
    if pd.isna(value):
        return None
    
    value_part = value.split(':')[-1].strip() if ':' in value else value.strip()
    
    try:
        return float(value_part)
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    value_part = value.split(':')[-1].strip().lower() if ':' in value else value.lower().strip()
    
    if 'female' in value_part or 'f' == value_part:
        return 0
    elif 'male' in value_part or 'm' == value_part:
        return 1
    return None

# 3. Save Metadata
# Determine trait availability based on trait_row
is_trait_available = trait_row is not None

# Use the validate_and_save_cohort_info function to save information
# Since this is not the final validation, we use is_final=False
_ = validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None, indicating clinical data is not available
# If trait_row is not None, we would have done:
# clinical_data = pd.read_csv(...)  # Load clinical data
# clinical_df = geo_select_clinical_features(
#     clinical_df=clinical_data, 
#     trait=trait, 
#     trait_row=trait_row, 
#     convert_trait=convert_trait,
#     age_row=age_row, 
#     convert_age=convert_age,
#     gender_row=gender_row, 
#     convert_gender=convert_gender
# )
# preview = preview_df(clinical_df)
# print(preview)
# clinical_df.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 74
Header line: "ID_REF"	"GSM977688"	"GSM977689"	"GSM977690"	"GSM977691"	"GSM977692"	"GSM977693"	"GSM977694"	"GSM977695"	"GSM977696"	"GSM977697"	"GSM977698"	"GSM977699"	"GSM977700"	"GSM977701"	"GSM977702"	"GSM977703"	"GSM977704"	"GSM977705"	"GSM977706"	"GSM977707"	"GSM977708"	"GSM977709"	"GSM977710"	"GSM977711"	"GSM977712"	"GSM977713"	"GSM977714"	"GSM977715"	"GSM977716"	"GSM977717"	"GSM977718"	"GSM977719"	"GSM977720"	"GSM977721"	"GSM977722"	"GSM977723"	"GSM977724"	"GSM977725"	"GSM977726"	"GSM977727"	"GSM977728"	"GSM977729"	"GSM977730"	"GSM977731"	"GSM977732"	"GSM977733"	"GSM977734"	"GSM977735"	"GSM977736"	"GSM977737"	"GSM977738"	"GSM977739"	"GSM977740"	"GSM977741"	"GSM977742"	"GSM977743"	"GSM977744"	"GSM977745"	"GSM977746"	"GSM977747"	"GSM977748"	"GSM977749"	"GSM977750"
First data line: "10000-V3-70mer-rc"	0.0001741	0.0103851	0.0018561	0.0001921	0.0000011	-0.0000001	0.0173041	0.0000021	0.0000031	0.0000141	0.0000021	0.0000001	0.0105871	0.0224131	0.0129061	-0.00

### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers examined from the provided data, 
# these appear to be custom microarray probe IDs ("10000-V3-70mer-rc", "10001-V3-70mer-rc", etc.)
# rather than standard human gene symbols like BRCA1, TP53, etc.
# These identifiers need to be mapped to human gene symbols for proper interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1-V3-70mer', '10-V3-70mer', '100-V3-70mer', '1000-V3-70mer', '10000-V3-70mer-rc'], 'SEQUENCE': ['ATTGCGGTCAATCAAGAGATTGGATTCAAAGACTTGGTCTAGAGCTCCATCTAATAAGCATACATTTTTA', 'AAACTAACAGATCACGACCCTATAGTAAAGAAGCCTAAATTATCTGAAAAAACTCTCCTTCACCTAGTAA', 'ATTAATTTCTCGTAAAAGTAGAAAATATATTCTAATTATTGCACGGTAAGGAAGTAGATCATAAAGAACA', 'GCATAAGTGCTCGCAATGATGTAGCTGCTTACGCTTGCTTACTCCGCCCTGAAACGCCTACCTTAAACGC', 'GCAAAAAGCGCGTTAACAGAAGCGAGAAGCGAGCTGATTGGTTAGTTTAAATAAGGCTTGGGGTTTTTCC'], 'ProbeName': ['1-V3-70mer', '10-V3-70mer', '100-V3-70mer', '1000-V3-70mer', '10000-V3-70mer-rc'], 'GeneName': ['V3-6063472_104-70mer-Parvo-like-virus-Parvoviridae-Parvovirus', 'V3-1113784_116-70mer-Muscovy-duck-parvovirus-Parvoviridae-Dependovirus', 'V3-297378_9-70mer-Vaccinia-virus-Poxviridae-Orthopoxvirus', 'V3-210706_327-70mer-Bovine-immunodeficiency-virus-Retroviridae-Lentivirus', 'V3-9628654_322_rc-70mer-Murine-type-C-retrovirus-Retroviridae-Gammaretrovirus']}


### Step 6: Gene Identifier Mapping

In [7]:
# Examining the gene identifiers and annotations
# The gene expression data has identifiers like '10000-V3-70mer-rc'
# The gene annotation has columns 'ID' (with values like '10000-V3-70mer-rc') and 'GeneName' with virus names

# 1. Identify the columns for mapping
# The column 'ID' in gene_annotation corresponds to probe identifiers in gene_data
# The 'GeneName' column contains information about viral sequences, not human gene symbols

# Looking at the data: these are viral microarray probes (Virochip) - not human gene expression
# This is consistent with the study design which is about detecting XMRV virus, not human gene expression

# 2. Create gene mapping dataframe - we'll use 'ID' and 'GeneName' columns
# However, we should note that this dataset doesn't actually contain human gene expression data
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'GeneName')
print("Gene mapping preview:")
print(preview_df(mapping_data))

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# Note: This will map viral probes to viral genes, not human genes
gene_data = apply_gene_mapping(gene_data, mapping_data)
print("Mapped gene expression data preview (first 5 genes, first 5 samples):")
gene_data_preview = gene_data.iloc[:5, :5]
print(gene_data_preview)

# Save the gene data for completeness, though this is viral sequence data, not human gene expression
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)

# Final validation
# Since we've confirmed this isn't human gene expression data, we should update our records
is_gene_available = False
is_trait_available = trait_row is not None
is_biased = True  # Dataset is biased/not suitable as it contains viral data not human gene expression

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,  # Set to True to indicate dataset is not suitable
    df=gene_data,  # Use actual dataframe to preserve sample size info
    note="Dataset contains viral probe data (Virochip) for XMRV virus detection, not human gene expression data."
)

Gene mapping preview:
{'ID': ['1-V3-70mer', '10-V3-70mer', '100-V3-70mer', '1000-V3-70mer', '10000-V3-70mer-rc'], 'Gene': ['V3-6063472_104-70mer-Parvo-like-virus-Parvoviridae-Parvovirus', 'V3-1113784_116-70mer-Muscovy-duck-parvovirus-Parvoviridae-Dependovirus', 'V3-297378_9-70mer-Vaccinia-virus-Poxviridae-Orthopoxvirus', 'V3-210706_327-70mer-Bovine-immunodeficiency-virus-Retroviridae-Lentivirus', 'V3-9628654_322_rc-70mer-Murine-type-C-retrovirus-Retroviridae-Gammaretrovirus']}
Mapped gene expression data preview (first 5 genes, first 5 samples):
       GSM977688  GSM977689  GSM977690  GSM977691  GSM977692
Gene                                                        
A-      0.001185   0.005649   0.019542   0.028514   0.037372
A118-   0.000032   0.000058   0.000126   0.000071   0.000055
A59-    0.000014   0.000142   0.000321   0.000595   0.000247
B-      0.000694   0.006662   0.011163   0.015327   0.007441
B19-    0.000125   0.000253   0.002043   0.001312   0.000592
